In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [2]:
# hyperparameters
# (my GPU ran out of memory with the default settings - so i am halving all the parameters that affect model size)
# (the model size was 10.788929 M parameters)
# (the message was: OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 4.00 GiB total capacity;
# 3.23 GiB already allocated; 0 bytes free; 3.40 GiB reserved in total by PyTorch)
# If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
# See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF)
# Even with the reduced params (with 2.715713 M parameters) it ran out of memory until I restarted the kernel
# The training took 16 seconds on my GPU and 220 seconds on my CPU with one hundredth of the batches and half for all the footprints stats.

batch_size = 1  ## 32  ## 64 # how many independent sequences will we process in parallel?
block_size = 64  ## 128  ## 256 # what is the maximum context length for predictions?
max_iters = 1  ## 50  ## 5000  # how many training batches to do
eval_interval = 5  ## 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20  ## 200  # for estimate_loss
n_embd = 192  ## 384
n_head = 6  # giving head_size = n_embd // n_head = 32
n_layer = 1
dropout = 0.2

In [3]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
f'{vocab_size = }'

'vocab_size = 65'

In [4]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [7]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [8]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [9]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [10]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()), ' parameters')
start = time.time()

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

end = time.time()
print('time = ', end - start)

481985  parameters
step 0: train loss 4.2660, val loss 4.2757
time =  1.7603223323822021


In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


f,P$-LXqVaPZuMBK3n; W.eV3:sOklDBWV'HhL3mAD,EP?LbUghr VQRXS;?rJZcE S.pjXmdCv:GXZPunjAvwvF
Gs?bpmxGjmGCcBP
cLTqoL.Qgcc;KZFB!,U!cCcFJvillinwgX&tF3oWMJRj.Q DAzEHb:q&csRupoGLFFc.CBRx?T '-.GdKjChH'NG:cmXw!ooYyTj $.P,LH$&arnH3v'l,k'gmfdm
&PgXIBsXm
DLSWl:l
QR.CXJcmjbs'mgHEhe
UYWywf3Tm:Sv?gl.rlb!;TQb
dljPiXuV?
b,P;LAUwlEiPadb.cF-H,r'kLjo
 Xm3m,jc ,M:,DzRLV!FzP
&&WZMhh m
BjqqBjFo,LrOQZ&vVp FMoSuR3PGPvIoRKFc&nimChpWKSnHaclVDeEiPAZf 3bjHg3yjhGOKYyqYJNRfpU,m$DfL H,l&kFJeo!qcRkmfNcRVxZFqMMlgmxXBvoZICWFu:gXhkI


In [13]:
m.blocks[0]

Block(
  (sa): MultiHeadAttention(
    (heads): ModuleList(
      (0-5): 6 x Head(
        (key): Linear(in_features=192, out_features=32, bias=False)
        (query): Linear(in_features=192, out_features=32, bias=False)
        (value): Linear(in_features=192, out_features=32, bias=False)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
    (proj): Linear(in_features=192, out_features=192, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (ffwd): FeedFoward(
    (net): Sequential(
      (0): Linear(in_features=192, out_features=768, bias=True)
      (1): ReLU()
      (2): Linear(in_features=768, out_features=192, bias=True)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (ln1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
)

In [14]:
type(m)

__main__.GPTLanguageModel

In [15]:
for p in m.blocks[0].parameters(recurse=True):
    print(p.numel(), p.size())

6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
6144 torch.Size([32, 192])
36864 torch.Size([192, 192])
192 torch.Size([192])
147456 torch.Size([768, 192])
768 torch.Size([768])
147456 torch.Size([192, 768])
192 torch.Size([192])
192 torch.Size([192])
192 torch.Size([192])
192 torch.Size([192])
192 torch.Size([192])
